In [19]:
import pandas as pd 
import ast
import re
import numpy as np
df = pd.read_csv("./data/kickstarter_test.csv")
df.head(3)
df_copy = df.copy()

In [20]:
df.shape

(4535, 15)

In [21]:
COUNT = 0

def increment():
    global COUNT
    COUNT = COUNT+1


def clean_rewards(reward_list):
    try:
        tiers = ast.literal_eval(reward_list)
    except SyntaxError as e:
        increment()
        i = len(reward_list) -1
        while reward_list[i] != "{":
            i -= 1
        reward_list = reward_list[:(i-1)]
        reward_list += ']'
    return reward_list

In [22]:
#Clean rewards to remove broken off rewards
df_copy["rewards"] = df_copy["rewards"].apply(lambda x : clean_rewards(x))

df_copy["reward_tiers"] = df_copy["rewards"].apply(lambda x : len(ast.literal_eval(x)))


In [163]:
# Rearange reward_tiers column to beside rewards
cols = df_copy.columns.tolist()
cols = cols[:6] + [cols[-1]] + cols[6:-1]
df_copy = df_copy[cols]
df_copy

,id,name,description,description_story,description_risks,rewards,reward_tiers,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at
0,6.575370e+08,Barbells Over Meds,Webinar presented to teach step by step how I ...,My Story When I was 19 years old I left everyt...,NaN,"[{""rewards"":""Pledge US$ 55 or more\n\nAbout S$...",2,"{""id"":360,""name"":""Video"",""analytics_name"":""Vid...",200.0,2017-12-22 01:15:13,"{""id"":2383660,""name"":""Columbus"",""slug"":""columb...",0,0,NaN,2017-12-07 01:15:13,2017-12-01 01:22:21
1,3.598449e+08,Contemporary Circus Show,"An original, evening length, story-based circu...",Yes Ma'am Circus was created in January 2018 b...,"Most of the acts and costumes are ready, the s...","[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",13,"{""id"":254,""name"":""Performances"",""analytics_nam...",200.0,2018-04-27 13:42:17,"{""id"":2379574,""name"":""Chicago"",""slug"":""chicago...",1,0,https://v2.kickstarter.com/1663775949-NZlO%2FJ...,2018-03-13 13:42:17,2018-03-08 01:37:55
2,5.542695e+08,The Fur Trap book & record,Teenage rebels! Rock'n'roll impersonation! Big...,Lonely teenager Beth feels trapped when her ne...,"Thankfully, the book and soundtrack are (almos...","[{""rewards"":""Pledge £1 or more\n\nAbout US$ 2\...",11,"{""id"":45,""name"":""Art Books"",""analytics_name"":""...",3000.0,2013-08-10 07:45:40,"{""id"":44418,""name"":""London"",""slug"":""london-gb""...",1,1,https://v2.kickstarter.com/1664630049-IKu2RFTx...,2013-07-05 07:45:40,2013-03-05 20:13:03
3,2.966406e+08,Solidifi,I want to create a family friendly makerspace ...,This campaign is to create a makerspace called...,"Well, of couse, one of the biggest risks, as w...","[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",8,"{""id"":362,""name"":""Makerspaces"",""analytics_name...",12000.0,2015-01-12 05:22:26,"{""id"":2513768,""name"":""Walnut Creek"",""slug"":""wa...",0,0,NaN,2014-11-13 05:22:26,2014-10-31 03:36:38
4,5.871292e+08,The Za Report: Baking the Pizza Dream,I want to open my own brick and mortar AND bec...,"So we have hit the minimum goal of $50,000, th...",Thank you for supporting my Kickstarter! I hop...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout S$...",11,"{""id"":312,""name"":""Restaurants"",""analytics_name...",50000.0,2021-12-08 11:15:21,"{""id"":12589335,""name"":""Brooklyn"",""slug"":""brook...",1,1,https://v2.kickstarter.com/1664037271-%2FkfCyA...,2021-11-08 11:15:21,2021-08-15 13:32:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530,1.525744e+09,Waikīkī,Sundance Institute Native Lab & Feature Film P...,"An unflinching glimpse into ""paradise,” where ...",Film is an inherently risky business. An indie...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",27,"{""id"":293,""name"":""Drama"",""analytics_name"":""Dra...",31400.0,2018-04-03 06:00:00,"{""id"":2423945,""name"":""Honolulu"",""slug"":""honolu...",1,1,https://v2.kickstarter.com/1663902740-HkTbtIa2...,2018-03-02 19:45:58,2017-07-25 01:25:04
4531,9.913245e+07,a New Album from Ethan Lipton & his Orchestra,Micro Produce for our new CD! We're asking fr...,"Friends, we are making a new album. It will be...",NaN,"[{""rewards"":""Pledge US$ 15 or more\n\nAbout S$...",9,"{""id"":41,""name"":""Jazz"",""analytics_name"":""Jazz""...",12500.0,2009-11-09 17:00:00,the United States,1,1,NaN,2009-09-09 01:58:23,2009-09-01 19:02:42
4532,1.851292e+09,a series of illustrations to Dante's Inferno,to be published as an illustrated book,"Artists including William Blake, Botticelli, a...",I have completed the art work for this project...,"[{""rewards"":""Pledge US$ 50 or more\n\nAbout S$...",6,"{""id"":23,""name"":""Painting"",""analytics_name"":""P...",5000.0,2021-11-09 06:12:00,"{""id"":2367105,""name"":""Boston"",""slug"":""boston-m...",1,0,https://v2.kickstarter.com/1664137000-S5aNzkGv...,2021-10-09 18:27:38,2021-09-05 18:19:48
4533,1.827572e+09,Industrial 3D Printer. Closed Loop. Open Sourc...,"We are proud to offer the f

In [154]:
df_copy.iloc[4340, 5]

'[{"rewards":"Pledge $5 or more\\n\\nAbout $5\\n\\n\\n\\n👻 The Case of the Holiday Spirit\\n\\n\\nYour generous support is no mystery to us! We’ll publicly thank you in the theater lobby & show credits for your holiday charity.\\nIncludes:\\n\\n\\n🙏 Thank You in the Lobby & Credits\\n\\n\\n\\n\\nLess\\n\\n\\n\\n\\nEstimated delivery\\nNov 2022\\n\\n\\n\\n\\n\\n\\n85 backers"},{"rewards":"Pledge $15 or more\\n\\nAbout $15\\n\\n\\n\\n🎄 Santeeees Lil Helper\\n\\n\\nWax those ornaments & string garland on those boogie boards because Christmas is coming early this year! Join TCB & the Squad for a backer-only Christmas In July-ivestream: featuring songs, games, and revelry.\\nIncludes:\\n\\n\\n⛱  Christmas in July-ivestream\\n\\n\\n🙏 Thank You in the Lobby & Credits\\n\\n\\n\\n\\nLess\\n\\n\\n\\n\\nEstimated delivery\\nJul 2022\\n\\n\\n\\n\\n\\n\\n103 backers"},{"rewards":"Pledge $25 or more\\n\\nAbout $25\\n\\n\\n\\n👾  Festival of Bytes\\n\\n\\nWhether you’re counting down the days to a mir

Generate all reward amounts for each listing

In [164]:
# for i in range(len(df_copy)):
# money = re.compile('|'.join([
#   r'^\$?(\d+, \d{1,3})$',  # e.g., $.50, .50, $1.50, $.5, .5
#   r'^\$?(\d+)$',           # e.g., $500, $5, 500, 5
# ]))
df_copy["min_reward"] = 0
df_copy["max_reward"] = 0

for i in range(len(df_copy)):
    all_reward_amount = []
    reward = df_copy.iloc[i, 5]
    dict_list = ast.literal_eval(reward)
    # print(dict_list)
    for dictionary in dict_list:
        # print("dictionary item is ", dictionary)

        # if not isinstance(dictionary,  dict):
        #     print("ERROR type is", type(dict))
        values_string = str(dictionary.values())
        # reward_title_USD = re.search(r"(US\$\s)?(\$\s)?\d{1,3}(,\d{1,3})?", values_string) #US$ or $
        reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
        reward_title_USD = reward_title_SGD
        # reward_title = re.search(r"Pledge US\$ \d{1,3}(,\d{1,3})? or more", values_string)
        if reward_title_SGD or reward_title_USD:
            if reward_title_SGD is not None:
                # print("reward USD title is ", reward_title_USD.group())
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_USD.group())
            else:
                print("ERROR NO SGD")
                # print("reward SGD title is ", reward_title_SGD.group())
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
        else:
            # print("reward title is none")
            pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
            if pledged_without_reward:
                all_reward_amount.append(0)
    # print("ALL REWARD AMOUNT ", all_reward_amount)
    if len(all_reward_amount) > 0:
        df_copy["min_reward"][i] = min(all_reward_amount)
        df_copy["max_reward"][i] = max(all_reward_amount)
    

C:\Users\bandy\AppData\Local\Temp\ipykernel_21828\2523479555.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["min_reward"][i] = min(all_reward_amount)
C:\Users\bandy\AppData\Local\Temp\ipykernel_21828\2523479555.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy["max_reward"][i] = max(all_reward_amount)


In [185]:
# Rewards should be sorted in ascending order, any amount less than the max means reward is no longer available
result2[result2['min_reward'] == 0]

,id,name,description,description_story,description_risks,rewards,reward_tiers,min_reward,max_reward,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at
320,1.358511e+09,LED Floaties at Hoosac River Lights!,We want to give out hundreds of multi-colored ...,At last year's Hoosac River Lights festival hu...,NaN,[],0,0,0,"{""id"":53,""name"":""Public Art"",""analytics_name"":...",315.0,2009-08-15 03:55:00,"{""id"":2460561,""name"":""North Adams"",""slug"":""nor...",1,1,NaN,2009-07-29 17:41:55,2009-07-29 15:25:08
1348,1.414593e+09,Natural Vegan - Cosmetics SO NATURAL you can E...,The most natural bodycare products in the worl...,NEW RELEASE: We have hit our goal of $4800 in ...,We've face our biggest challenges yet. Our big...,"[{""rewards"":""Pledge CA$ 10 or more\n\nAbout S$...",30,0,325,"{""id"":315,""name"":""Vegan"",""analytics_name"":""Veg...",4800.0,2017-12-20 21:51:20,"{""id"":9807,""name"":""Vancouver"",""slug"":""vancouve...",1,0,https://v2.kickstarter.com/1664567040-dQCzRngn...,2017-11-20 21:51:20,2017-11-06 19:44:28
6154,1.392677e+09,The Binary Project | Robin Eley gives away a p...,Own part of a Robin Eley painting for FREE (+p...,THESIS As an artist and first time father of...,DELIVERY\r\n\r\nEffectively distributing 4900 ...,"[{""rewards"":""Pledge AU$ 0 or more\n\nAbout S$ ...",6,0,380,"{""id"":23,""name"":""Painting"",""analytics_name"":""P...",7000.0,2017-01-11 19:56:36,"{""id"":1099805,""name"":""Adelaide"",""slug"":""adelai...",1,0,https://v2.kickstarter.com/1664105839-ldRBH0KK...,2016-12-12 19:56:36,2016-12-01 17:36:08
10414,1.760127e+09,North Carolina Recovery All Ways Podcast,A podcast geared towards educating the general...,NCR is a team of recovery advocates with a mi...,NCR faces the challenges of audience expansion...,"[{""rewards"":""Pledge US$ 0 or more\n\nAbout S$ ...",8,0,1000,"{""id"":357,""name"":""Audio"",""analytics_name"":""Aud...",8000.0,2018-06-29 00:17:09,"{""id"":2388150,""name"":""Cullowhee"",""slug"":""cullo...",1,0,https://v2.kickstarter.com/1664006596-aXmhEiyq...,2018-05-30 00:17:09,2018-04-14 14:17:08
14736,1.693361e+09,Minimalist Bourbon Barrel Pens for EDC - Make 100,"This is a short, 20-day project for awesome mi...",This is a Make 100 project -- no more than 100...,I see no real risks with the supplies of this ...,"[{""rewards"":""Pledge US$ 50 or more\n\nAbout S$...",9,0,60,"{""id"":356,""name"":""Woodworking"",""analytics_name...",500.0,2018-01-22 18:04:29,"{""id"":2352532,""name"":""Alabaster"",""slug"":""alaba...",1,0,https://v2.kickstarter.com/1664014250-c4fiNqjj...,2018-01-02 18:04:29,2017-10-25 13:39:05
25063,1.495813e+09,Shane' Marie Ep,This is a RnB/Pop Extented Play (EP) Project F...,This is a RnB/Pop Extented Play (EP) Project F...,Im opening up my LIFE and ARTS to the world wi...,"[{""rewards"":""Pledge US$ 0 or more\n\nAbout S$ ...",2,0,100,"{""id"":322,""name"":""R&B"",""analytics_name"":""R&B"",...",1000.0,2015-11-21 15:50:31,"{""id"":2436398,""name"":""Lanham"",""slug"":""lanham-m...",0,0,https://v2.kickstarter.com/1664720668-H%2F3C6z...,2015-10-07 14:50:31,2015-10-01 17:02:33


In [65]:
df_copy.head()

,id,name,name.1,description,description_story,description_story.1,reward_tiers,description_risks,description_risks.1,rewards,...,deadline,location,state,state.1,staff_pick,video,launched_at,created_at,min_reward,max_reward
0,657537019.0,Barbells Over Meds,Barbells Over Meds,Webinar presented to teach step by step how I ...,NaN,My Story When I was 19 years old I left everyt...,2,NaN,NaN,"[{""rewards"":""Pledge US$ 55 or more\n\nAbout S$...",...,2017-12-22 01:15:13,"{""id"":2383660,""name"":""Columbus"",""slug"":""columb...",0,failed,0,NaN,2017-12-07 01:15:13,2017-12-01 01:22:21,55,110
1,359844914.0,Contemporary Circus Show,Contemporary Circus Show,"An original, evening length, story-based circu...",Story\n\n\n\n\n\n\n\nOur StoryYes Ma'am Circus...,Yes Ma'am Circus was created in January 2018 b...,13,"Most of the acts and costumes are ready, the s...","Most of the acts and costumes are ready, the s...","[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",...,2018-04-27 13:42:17,"{""id"":2379574,""name"":""Chicago"",""slug"":""chicago...",1,successful,0,https://v2.kickstarter.com/1663775949-NZlO%2FJ...,2018-03-13 13:42:17,2018-03-08 01:37:55,1,150
2,554269494.0,The Fur Trap book & record,The Fur Trap book & record,Teenage rebels! Rock'n'roll impersonation! Big...,Story\n\n\n\n\n\n\n\nThe StoryLonely teenager ...,Lonely teenager Beth feels trapped when her ne...,11,"Thankfully, the book and soundtrack are (almos...","Thankfully, the book and soundtrack are (almos...","[{""rewards"":""Pledge £1 or more\n\nAbout US$ 2\...",...,2013-08-10 07:45:40,"{""id"":44418,""name"":""London"",""slug"":""london-gb""...",1,successful,1,https://v2.kickstarter.com/1664630049-IKu2RFTx...,2013-07-05 07:45:40,2013-03-05 20:13:03,2,558
3,296640551.0,Solidifi,Solidifi,I want to create a family friendly makerspace ...,StoryThis campaign is to create a makerspace c...,This campaign is to create a makerspace called...,8,"Well, of couse, one of the biggest risks, as w...","Well, of couse, one of the biggest risks, as w...","[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",...,2015-01-12 05:22:26,"{""id"":2513768,""name"":""Walnut Creek"",""slug"":""wa...",0,failed,0,NaN,2014-11-13 05:22:26,2014-10-31 03:36:38,1,10000
4,587129161.0,The Za Report: Baking the Pizza Dream,The Za Report: Baking the Pizza Dream,I want to open my own brick and mortar AND bec...,NaN,"So we have hit the minimum goal of $50,000, th...",11,NaN,Thank you for supporting my Kickstarter! I hop...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout S$...",...,2021-12-08 11:15:21,"{""id"":12589335,""name"":""Brooklyn"",""slug"":""brook...",1,successful,1,https://v2.kickstarter.com/1664037271-%2FkfCyA...,2021-11-08 11:15:21,2021-08-15 13:32:08,10,10000


In [ ]:
# Rearange all_reward_amount column to beside reward_tiers
cols = df_copy.columns.tolist()
cols = cols[:7] + cols[-2:] + cols[7:-1]
df_copy = df_copy[cols]


In [2]:
# Remove rewards that got cut off while scraping and closes the dictionary
def clean_rewards(reward_list):
    try:
        tiers = ast.literal_eval(reward_list)
    except SyntaxError as e:
        i = len(reward_list) -1
        while reward_list[i] != "{":
            i -= 1
        reward_list = reward_list[:(i-1)]
        reward_list += ']'
    return reward_list
    
def create_rewards_tiers(df):
    df["rewards"] = df["rewards"].apply(lambda x : clean_rewards(x))
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_min_max_reward(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        all_reward_amount = []
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        for dictionary in dict_list:
            values_string = str(dictionary.values())
            # reward_title_USD = re.search(r"(US\$\s)?(\$\s)?\d{1,3}(,\d{1,3})?", values_string) #US$ or $
            reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$

            if reward_title_SGD:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())
            else:
                 print("ERROR NO SGD")   

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
            else:
                pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
                if pledged_without_reward:
                    all_reward_amount.append(0)
        if len(all_reward_amount) > 0:
            df["min_reward"][i] = min(all_reward_amount)
            df["max_reward"][i] = max(all_reward_amount)
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange min and max reward columns to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-2]
    df = df[cols]
    return df

In [3]:
result = create_rewards_tiers(df_copy)
result.head()

,id,name,description,description_story,description_risks,rewards,reward_tiers,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at
0,1.193757e+09,DARKARTS ALBUM,Help DARKARTS record their next album!,To the genuine music fans! Welcome to DARKARTS...,We anticipate few risks to launching this proj...,"[{""rewards"":""Pledge US$ 25 or more\n\nAbout S$...",6,"{""id"":40,""name"":""Indie Rock"",""analytics_name"":...",3400.0,2022-06-11 21:01:43,"{""id"":2391279,""name"":""Denver"",""slug"":""denver-c...",1,0,https://v2.kickstarter.com/1664695127-1NAeIZwv...,2022-05-12 21:01:43,2022-03-24 02:02:28
1,1.802286e+09,100% Hand-Poured Soy Candles w/ Wood Wicks in ...,I hand-pour in small batches 8oz. 100% soy can...,I am looking to raise funds in order to buy pe...,The only challenges that I can foresee would b...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",6,"{""id"":343,""name"":""Candles"",""analytics_name"":""C...",1000.0,2016-01-20 23:00:54,"{""id"":2502716,""name"":""Sussex"",""slug"":""sussex-n...",1,0,NaN,2015-12-21 23:00:54,2015-12-21 18:36:30
2,9.537778e+07,First Move by Ori Flomin,First Move is a reflection on the mature danci...,First\nMove is a reflection on the mature danc...,The risks and challenges are the same as those...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout S$...",9,"{""id"":254,""name"":""Performances"",""analytics_nam...",7500.0,2015-03-13 16:09:51,"{""id"":2459115,""name"":""New York"",""slug"":""new-yo...",1,1,https://v2.kickstarter.com/1663849120-vK6gi3cV...,2015-02-11 17:09:51,2015-01-18 09:26:16
3,3.148434e+08,Looking for the Masters in Ricardo's Golden Shoes,A beautiful hardcover photo book with gold edg...,"One summer day in 2013, I was having breakfast...",The risks and challenges when publishing a pho...,"[{""rewards"":""Pledge €2 or more\n\nAbout S$ 3\n...",9,"{""id"":276,""name"":""Fine Art"",""analytics_name"":""...",12000.0,2016-04-24 11:20:25,"{""id"":615702,""name"":""Paris"",""slug"":""paris-fr"",...",1,1,https://v2.kickstarter.com/1664123995-bH2MjakM...,2016-03-24 11:20:25,2016-03-18 22:57:42
4,1.655207e+08,Deep Love: Going to the Moon,Deep Love is headed to the moon! (Via Oregon) ...,********** Stretch Goals************ We're com...,As we learned with our last kickstarter campai...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout US$...",22,"{""id"":284,""name"":""Musical"",""analytics_name"":""M...",5000.0,2014-08-31 12:54:48,"{""id"":2487610,""name"":""Salt Lake City"",""slug"":""...",1,1,https://v2.kickstarter.com/1664440295-pSizLCYF...,2014-08-01 12:54:48,2014-03-28 22:42:12


In [4]:
result2 = create_min_max_reward(result)
result2.tail()

C:\Users\bandy\AppData\Local\Temp\ipykernel_3592\1520128763.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["min_reward"] = 0
C:\Users\bandy\AppData\Local\Temp\ipykernel_3592\1520128763.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["max_reward"] = 0
C:\Users\bandy\AppData\Local\Temp\ipykernel_3592\1520128763.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,id,name,description,description_story,description_risks,rewards,reward_tiers,min_reward,max_reward,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at
25687,1.441372e+09,Halloween Haunt 2014,I build a famous halloween haunt for my neighb...,Every year I have built a halloween haunt usin...,The biggest challenge is meeting the strict de...,"[{""rewards"":""Pledge US$ 10 or more\n\nAbout S$...",3,10,100,"{""id"":260,""name"":""Interactive Design"",""analyti...",1500.0,2014-09-26 07:02:19,"{""id"":2523270,""name"":""Woodland Hills"",""slug"":""...",1,0,NaN,2014-08-27 07:02:19,2014-08-04 01:47:34
25688,7.841194e+08,Affordable & Customizable Made-to-Measure Dresses,Women shouldn't have to make themselves fit to...,"At ELSA AND ME, we believe clothing should b...","As with most made-to-order products, there are...","[{""rewards"":""Pledge US$ 15 or more\n\nAbout S$...",10,15,2500,"{""id"":263,""name"":""Apparel"",""analytics_name"":""A...",10000.0,2016-10-13 10:02:08,"{""id"":12589335,""name"":""Brooklyn"",""slug"":""brook...",1,1,https://v2.kickstarter.com/1663822418-UFq0ddpU...,2016-09-13 10:02:08,2016-09-09 16:21:07
25689,1.889404e+09,My Own Soda Kiln,I just finished my MFA in ceramics and to move...,Well guys here’s what's going on. I finished ...,I'm lucky to have a community of potters and m...,"[{""rewards"":""Pledge US$ 1 or more\n\nAbout S$ ...",29,1,1000,"{""id"":350,""name"":""Pottery"",""analytics_name"":""P...",10000.0,2014-07-18 22:39:11,"{""id"":2503317,""name"":""Sylva"",""slug"":""sylva-nc""...",1,1,https://v2.kickstarter.com/1664213566-o9yLnQFU...,2014-06-18 22:39:11,2014-04-08 01:33:08
25690,9.875153e+08,Echoes from Iron Ore,Recorded at the edge of everywhere,"Over three days in March 2021, a group of nine...",A phrase repeated often during the cabin recor...,"[{""rewards"":""Pledge US$ 15 or more\n\nAbout S$...",6,15,4500,"{""id"":316,""name"":""Blues"",""analytics_name"":""Blu...",9000.0,2021-04-29 14:23:15,"{""id"":2487129,""name"":""St. Paul"",""slug"":""st-pau...",1,0,https://v2.kickstarter.com/1664734147-PMZFYJnr...,2021-03-30 14:23:15,2021-03-29 10:58:55
25691,1.405322e+09,The Cryptozoologist,"A Yorkshire-based, one-woman yarn dyeing busin...",The Cryptozoologist is at the moment a kitchen...,The single biggest challenge is sourcing local...,"[{""rewards"":""Pledge £5 or more\n\nAbout S$ 8\n...",4,8,78,"{""id"":348,""name"":""Knitting"",""analytics_name"":""...",200.0,2014-06-21 16:10:26,"{""id"":12602193,""name"":""East Riding of Yorkshir...",1,0,NaN,2014-05-22 16:10:26,2014-05-20 21:27:51
